In [1]:
# Some sections of BERT code adapted from:
# https://github.com/google-research/bert# 
# https://www.kaggle.com/shaz13/spooky-author-identification-bert
# https://github.com/loretoparisi/bert-movie-reviews-sentiment-classifier/blob/master/src/bert_sentiment_classifier.ipynb

# Initial hyperparameters set using:
# https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb

In [2]:
# Uncomment the pip install lines in this cell to install packages you'll likely need to install. 
# The rest you likely have installed and if not are an easy pip install.
# If you already have a recent version tensorflow you'll need to remove it and downgrade to this one
#!pip install tensorflow==1.15.0
#!pip install bert-tensorflow
#!pip install tensorflow_hub

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import os
from math import e
import datetime as dt

In [4]:
OUTPUT_DIR = './BERT_checkpoints' 

In [5]:
input_data = pd.read_csv('reddit_data_raw_small.csv')

authormap = list(input_data['author'].unique())
authormap_num = list(range(len(authormap)))
authormap_dict = {"author":authormap, "authormap_num":authormap_num}
authormap_df = pd.DataFrame(authormap_dict)
authormap_df.head()

,author,authormap_num
0,Bouck,0
1,dortega209,1
2,IamHumanAndINeed,2
3,Pm-me-gift-cardz,3
4,lizardswillcontrolus,4


In [6]:
train_pre, test = train_test_split(input_data, test_size = 0.2, random_state = 100)
train, val =  train_test_split(train_pre, test_size = 0.2, random_state = 100)
train.head()

,author,comment,time,score,subreddit
6,Bouck,Thank you for your understanding. I wish you t...,1568203728,2,MurderedByWords
425,Pm-me-gift-cardz,Definitely looked a bit bigger than Al imo,1567867469,5,MMA
16,Bouck,His first tour with them was the one they just...,1568801292,1,thrice
252,IamHumanAndINeed,I always thought it was Crantist !,1568194406,1,europe
451,Pm-me-gift-cardz,Obviously your main opponent should you contin...,1568910995,1,IAmA


In [7]:
def preprocess(data, authormap=authormap_df):
    needed_cols = data[['author','comment']]
    new_data = needed_cols.merge(authormap, how='left', on='author')
    new_data = new_data.drop(['author'], axis=1)
    return(new_data)

train_processed = preprocess(train)
val_processed = preprocess(val)
train_processed.head()

,comment,authormap_num
0,Thank you for your understanding. I wish you t...,0
1,Definitely looked a bit bigger than Al imo,3
2,His first tour with them was the one they just...,0
3,I always thought it was Crantist !,2
4,Obviously your main opponent should you contin...,3


In [8]:
print("# Training Records :", len(train))
print("# Validation Records :", len(val))
print("# Testing Records :", len(test))

# Training Records : 377
# Validation Records : 95
# Testing Records : 119


In [9]:
DATA_COLUMN = 'comment'
LABEL_COLUMN = 'authormap_num'

train_InputExamples = train_processed.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = val_processed.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [10]:
# This is the path to BERT_base. Replace this link to change the BERT model
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
#BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-24_H-1024_A-16/1"

def create_tokenizer_from_hub_module():
    """Get the vocab file and casing info from the Hub module."""
    with tf.Graph().as_default():
        bert_module = hub.Module(BERT_MODEL_HUB)
        tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
        with tf.Session() as sess:
            vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                                tokenization_info["do_lower_case"]])
      
    return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [11]:
# Max number of tokens per comment
MAX_SEQ_LENGTH = 96

# Convert training and validation records to features BERT can digest
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, authormap_num, MAX_SEQ_LENGTH, tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, authormap_num, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 377


INFO:tensorflow:Writing example 0 of 377


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


In [12]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels, num_labels):
  
    bert_module = hub.Module(BERT_MODEL_HUB, trainable=True)
    bert_inputs = dict(input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids)
    bert_outputs = bert_module(inputs=bert_inputs, signature="tokens", as_dict=True)

    output_layer = bert_outputs["pooled_output"]

    hidden_size = output_layer.shape[-1].value

    output_weights = tf.get_variable("output_weights", [num_labels, hidden_size], 
                                     initializer=tf.truncated_normal_initializer(stddev=0.02))

    output_bias = tf.get_variable("output_bias", [num_labels], initializer=tf.zeros_initializer())

    with tf.variable_scope("loss"):

        # Dropout helps prevent overfitting
        output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

        logits = tf.matmul(output_layer, output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, output_bias)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        # Convert labels into one-hot encoding
        one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
        predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
        
        # If we're predicting, we want predicted labels and the probabiltiies.
        if is_predicting:
            return (predicted_labels, log_probs)

        # If we're train/eval, compute loss between predicted and actual label
        per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
        loss = tf.reduce_mean(per_example_loss)
        return (loss, predicted_labels, log_probs)

In [13]:
# A function that adapts the model to work for training, evaluation, and prediction.

def model_fn_builder(num_labels, learning_rate, num_train_steps,num_warmup_steps):
    # creates our model function
    
    def model_fn(features, labels, mode, params):  
        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
        # Training and eval
        if not is_predicting:

            (loss, predicted_labels, log_probs) = create_model(is_predicting, input_ids, input_mask, 
                                                               segment_ids, label_ids, num_labels)

            train_op = bert.optimization.create_optimizer(loss, learning_rate, num_train_steps, 
                                                          num_warmup_steps, use_tpu=False)

            # Calculate evaluation metrics. 
            def metric_fn(label_ids, predicted_labels):
                accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        
                return {
                    "eval_accuracy": accuracy,
                    }

            eval_metrics = metric_fn(label_ids, predicted_labels)

            if mode == tf.estimator.ModeKeys.TRAIN:
                return tf.estimator.EstimatorSpec(mode=mode,
                  loss=loss,
                  train_op=train_op)
            else:
                return tf.estimator.EstimatorSpec(mode=mode,
                loss=loss,
                eval_metric_ops=eval_metrics)
        else:
            (predicted_labels, log_probs) = create_model(
            is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

            predictions = {
              'probabilities': log_probs,
              'labels': predicted_labels
            }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # return the actual model function in the closure
    return model_fn

In [14]:
# Compute train and warmup steps from batch size
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1.0
WARMUP_PROPORTION = 0.05

# Model configs
SAVE_CHECKPOINTS_STEPS = 300
SAVE_SUMMARY_STEPS = 100

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS
)

In [15]:
#Initializing the model and the estimator
model_fn = model_fn_builder(
  num_labels=len(authormap_num),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': './BERT_checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002B1143E1548>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': './BERT_checkpoints', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002B1143E1548>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [16]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

# Create an input function for validating. drop_remainder = True for using TPUs.
val_input_fn = run_classifier.input_fn_builder(
    features=val_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [17]:
#Training the model
print(f'Training has begun')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Training has begun
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 11 into ./BERT_checkpoints\model.ckpt.


INFO:tensorflow:Saving checkpoints for 11 into ./BERT_checkpoints\model.ckpt.


INFO:tensorflow:Loss for final step: 1.2987496.


INFO:tensorflow:Loss for final step: 1.2987496.


Training took time  0:03:30.725706


In [18]:
#Evaluating the model with Validation set
estimator.evaluate(input_fn=val_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 11: eval_accuracy = 0.51578945, global_step = 11, loss = 1.3738505


INFO:tensorflow:Saving dict for global step 11: eval_accuracy = 0.51578945, global_step = 11, loss = 1.3738505


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 11: ./BERT_checkpoints\model.ckpt-11


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 11: ./BERT_checkpoints\model.ckpt-11


{'eval_accuracy': 0.51578945, 'loss': 1.3738505, 'global_step': 11}

In [19]:
# A method to get predictions
def getPrediction(in_sentences):
    #A list to map the actual labels to the predictions
    labels = list(authormap_df.author.values)

    #Transforming the test data into BERT accepted form
    input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] 

    #Creating input features for Test data
    input_features = run_classifier.convert_examples_to_features(input_examples, authormap_num, MAX_SEQ_LENGTH, tokenizer)

    #Predicting the classes 
    predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
    predictions = estimator.predict(predict_input_fn)
    return [(sentence, prediction['probabilities'],prediction['labels'], labels[prediction['labels']]) 
            for sentence, prediction in zip(in_sentences, predictions)]

In [20]:
predictions = getPrediction(list(test['comment']))

INFO:tensorflow:Writing example 0 of 119


INFO:tensorflow:Writing example 0 of 119


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from ./BERT_checkpoints\model.ckpt-11


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


In [21]:
for_df = [list(i[1]) for i in predictions]
bert_df = pd.DataFrame(for_df)
bert_df.head()

,0,1,2,3,4
0,-1.449637,-1.695423,-2.139853,-0.987188,-2.391115
1,-1.716760,-1.093279,-2.044376,-1.207311,-2.868648
2,-1.360213,-1.509751,-1.785650,-1.413603,-2.193951
3,-2.149933,-2.571558,-0.658841,-1.917778,-1.946954
4,-1.581853,-2.410993,-1.188470,-2.092483,-1.285138


In [22]:
reddit_df = test.reset_index().merge(bert_df, left_index = True, right_index=True)
reddit_df.head()

,index,author,comment,time,score,subreddit,0,1,2,3,4
0,391,Pm-me-gift-cardz,May not be upvoted as much but on the first wa...,1569184183,5,MMA,-1.449637,-1.695423,-2.139853,-0.987188,-2.391115
1,450,Pm-me-gift-cardz,"Theres a difference between this and trickery,...",1568918044,2,MMA,-1.716760,-1.093279,-2.044376,-1.207311,-2.868648
2,140,dortega209,Minshew leaves NFL for BangBros after winning ...,1569017980,10,nfl,-1.360213,-1.509751,-1.785650,-1.413603,-2.193951
3,240,IamHumanAndINeed,Ah merde il y a un bouton reset ?,1568924810,2,france,-2.149933,-2.571558,-0.658841,-1.917778,-1.946954
4,78,Bouck,Congrats?,1568370996,0,AppleWatch,-1.581853,-2.410993,-1.188470,-2.092483,-1.285138


In [23]:
reddit_data = reddit_df[map(lambda x :x not in ['author','comment'], list(reddit_df.columns))]
reddit_labels = reddit_df[['author']]

#convert subreddit to one-hot
dummies = pd.get_dummies(reddit_data.subreddit)
reddit_data = reddit_data.join(dummies)
reddit_data = reddit_data.drop(['subreddit'], axis=1)

#convert unix timestamp datetime, bin by hour and convert to one-hot
reddit_data['time'] = reddit_data['time'].apply(lambda x: dt.datetime.fromtimestamp(x).strftime('%H'))
reddit_data = reddit_data.drop(['time'], axis=1)

mms = MinMaxScaler()
reddit_data[['score']] = mms.fit_transform(reddit_data[['score']])
bert_cols = [i for i in range(0,len(reddit_labels.drop_duplicates()))]
reddit_data[bert_cols] = mms.fit_transform(reddit_data[bert_cols])


train_img, test_img, train_lbl, test_lbl = train_test_split(reddit_data, reddit_labels, test_size=0.2, random_state=100)

train_img.head()

,index,score,0,1,2,3,4,AppleWatch,AskReddit,AskTrumpSupporters,...,poppunkers,seinfeld,shittysuperpowers,technology,teenagers,television,thanosdidnothingwrong,thrice,todayilearned,worldnews
35,522,0.491935,0.766829,0.248389,0.440112,0.330280,0.863064,0,0,0,...,0,0,0,0,1,0,0,0,0,0
43,558,0.491935,0.516049,0.178503,0.551523,0.412617,0.866534,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73,97,0.500000,0.889533,0.772293,0.299936,0.464317,0.313859,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80,469,0.508065,0.822873,0.107834,0.519486,0.469725,0.753385,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,216,0.572581,0.736843,0.850387,0.235105,0.599219,0.135419,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
model = LogisticRegression(solver = 'lbfgs', max_iter=100)
model.fit(train_img, train_lbl)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: Convergen

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [25]:
y_pred = model.predict(test_img)
accuracy = metrics.accuracy_score(test_lbl.values, y_pred)
print('Classification Accuracy: {:.3f}'.format(accuracy))

Classification Accuracy: 0.958
